In [54]:
#2JKSRTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from decimal import Decimal
import pandas as pd
import time


driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20) 

bus_route_link = []
bus_route_name = []
bus_route_final = []
#bus_route = []
bus_names = [] 
bus_type = [] 
departing_time = []
duration = [] 
Reaching_time = [] 
star_rating = [] 
price = [] 
seat_availablity = []

def scroll(): # function to scroll page from top to bottom
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to the bottom in order to load all the buses
        time.sleep(2)# Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
        if new_height == last_height:# If it is the same height then it is at the end of the page
            break
        last_height = new_height

def click_button(): # function to click button for view buses for government buses
    view_buses = driver.find_elements(By.CSS_SELECTOR, '.p-left-10.icon.icon-down')
    if view_buses:
        button_count = len(view_buses)
        for i in range(button_count-1 , -1, -1):
            view_buses[i].click()

def get_bus_details(): # function to get all bus details
    rate = driver.find_elements(By.CSS_SELECTOR,".fare.d-block")#
    seat_available = driver.find_elements(By.CSS_SELECTOR,".column-eight.w-15.fl")
    #rating = driver.find_elements(By.CSS_SELECTOR,".column-six.p-right-10.w-10.fl")
    try:
        rating = driver.find_elements(By.CSS_SELECTOR, ".div.rating-sec.lh-24")
        rating = float(rating.text) if (rating.text) != "N/A" else 0.0
    except NoSuchElementException:
        rating = 0.0
            
    names = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
    b_type = driver.find_elements(By.CSS_SELECTOR,".bus-type.f-12.m-top-16.l-color.evBus")
    start_time = driver.find_elements(By.CSS_SELECTOR,".dp-time.f-19.d-color.f-bold")
    total_time = driver.find_elements(By.CSS_SELECTOR,".dur.l-color.lh-24")
    end_time =  driver.find_elements(By.CSS_SELECTOR,".bp-time.f-19.d-color.disp-Inline")
    
    time.sleep(5)
    print(len(names),len(b_type),len(start_time),len(total_time),len(end_time),len(rating),len(rate),len(seat_available))

    for i in range(0,len(seat_available)):#(len(rating)):
        bus_names.append(names[i].text)
        bus_type.append(b_type[i].text)
        departing_time.append(start_time[i].text)
        duration.append(total_time[i].text)
        Reaching_time.append(end_time[i].text)
        if (rating[i].text) == "New" or (rating[i].text) == "" or (rating[i].text) == None:
            star_rating.append(0.0)
        else:
            star_rating.append((rating[i].text)[0:3])
        price.append((rate[i].text))
        seat_availablity.append((seat_available[i].text)[0:2])
    
    print(len(bus_names),len(bus_type),len(departing_time),len(duration),len(Reaching_time),len(star_rating),len(price),len(seat_availablity))


driver.get("https://www.redbus.in/online-booking/jksrtc")
driver.maximize_window()
time.sleep(5)

#jump_pages()
find_bus_routes = driver.find_elements(By.CLASS_NAME,'route')
for route in find_bus_routes:
    route_name = route.get_attribute("title")
    route_link = route.get_attribute("href")
    bus_route_name.append(route_name)
    bus_route_link.append(route_link)
    
print("Total routes",len(bus_route_name))

for i in range (0,len(bus_route_link)):# used 2 just to check if every thing is working as expected
    driver.get(bus_route_link[i])
    body = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.clearfix.bus-item')))
    click_button()
    scroll()
    time.sleep(5)
    get_bus_details()
    #print(i,"Route completed")
    temp_name = []
    temp_link = []
    bus_route = []
    state_service = []
    print(len(bus_names))
    for j in range(len(bus_names)):
        temp_name.append(bus_route_name[i])
        temp_link.append(bus_route_link[i])
        state_service.append("JKSRTC")
    print(len(temp_name),len(temp_link))
    bus_route = list(zip(state_service,temp_name,temp_link,bus_names,bus_type,departing_time,duration,Reaching_time,star_rating,price,seat_availablity))
    bus_route_final.extend(bus_route)
    print(i+1,bus_route_name[i],"completed")
    temp_name.clear()
    temp_link.clear()
    state_service.clear()
    bus_route.clear()
    bus_names.clear()
    bus_type.clear()
    departing_time.clear()
    duration.clear()
    Reaching_time.clear()
    star_rating.clear()
    price.clear()
    seat_availablity.clear()
    
df_bus_details_2 = pd.DataFrame(bus_route_final, columns=["Service Name","Route_name", "Route_link", "Bus_names","Bus_type","Departing_time","Duration","Reaching_time","Star_rating","Price","Seat_availablity"])
df_bus_details_2['Seat_availablity']=df_bus_details_2['Seat_availablity'].astype(int)
df_bus_details_2['Price'] = df_bus_details_2['Price'].str.replace('INR ', '', regex=False)
#df_bus_details_2['Star_rating'] = df_bus_details_2['Star_rating'].str.replace('', '0.0', regex=False)
df_bus_details_2['Price'] = df_bus_details_2['Price'].astype(float)
df_bus_details_2['Star_rating'] = df_bus_details_2['Star_rating'].fillna(0.0)

Total routes 3


NameError: name 'NoSuchElementException' is not defined

In [48]:
df_bus_details_2

,Service Name,Route_name,Route_link,Bus_names,Bus_type,Departing_time,Duration,Reaching_time,Star_rating,Price,Seat_availablity
0,JKSRTC,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,A/C Sleeper (2+1),21:35,19h 30m,17:05,4.8\n9,2792.00,22
1,JKSRTC,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),20:00,19h 20m,15:20,5.0\n7,2684.00,32
2,JKSRTC,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,RAO TRAVELS INDIA PVT LTD.,Mercedes Benz A/C (2+2),16:00,19h 30m,11:30,2.2,3000.00,48
3,JKSRTC,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,JKRTC - 169882,A/C Seater Executive (2+1),07:30,01h 00m,08:30,1.2\n5,166.00,19
4,JKSRTC,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Bhawani Travels,A/C Sleeper (2+2),04:40,01h 50m,06:30,3.8,500.00,19
5,JKSRTC,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Moon Light Travels Katra,A/C Sleeper (2+2),05:00,02h 00m,07:00,4.1,552.00,38
6,JKSRTC,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),06:00,01h 30m,07:30,4.4,579.00,42
7,JKSRTC,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),06:15,04h 55m,11:10,4.4,286.00,18
8,JKSRTC,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),07:20,00h 50m,08:10,3.7,629.00,36
9,JKSRTC,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,City Land Travels,A/C Seater/Sleeper (2+1),09:30,01h 20m,10:50,3.5,360.00,34


In [56]:
#df_bus_details_2['Star_rating'] = df_bus_details_2['Star_rating'].astype(float)
df_bus_details_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Service Name      52 non-null     object 
 1   Route_name        52 non-null     object 
 2   Route_link        52 non-null     object 
 3   Bus_names         52 non-null     object 
 4   Bus_type          52 non-null     object 
 5   Departing_time    52 non-null     object 
 6   Duration          52 non-null     object 
 7   Reaching_time     52 non-null     object 
 8   Star_rating       52 non-null     object 
 9   Price             52 non-null     float64
 10  Seat_availablity  52 non-null     int32  
dtypes: float64(1), int32(1), object(9)
memory usage: 4.4+ KB


In [58]:
df_bus_details_2.to_csv("test.csv",index = False)